In [ ]:
import cv2 as cv
import numpy as np

### Preprocessing

#### Histogram balance

In [ ]:
def imageResizeByWidth(img, width=500):
    
    scale_percent = percent # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized = cv.resize(img, dim, interpolation = cv2.INTER_AREA)
    return resized

In [ ]:
def resizeImage(img, percent=100):
    scale_percent = percent # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized = cv.resize(img, dim, interpolation = cv.INTER_AREA)
    return resized

In [96]:
# Load the target image with the shapes we're trying to match
target = cv.imread('data/2.jpg')
target = resizeImage(target, 30)
target_gray = cv.cvtColor(target, cv.COLOR_BGR2GRAY)
target_gray = cv.medianBlur(target_gray, 9)

# cv.imshow('Target', target_gray)
# cv.waitKey()
# cv.destroyWindow('Target')

### Util function

In [5]:
def sharpImg(img):
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    kernel_sharpening = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    # applying different kernels to the input image
    return cv.filter2D(img_gray, -1, kernel_sharpening)

### Get green color from image

In [97]:
frame = target.copy()
# Convert BGR to HSV
hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
# cv.imshow("hsv", hsv)
# define range of blue color in HSV
lower = np.array([40,0,0])
upper = np.array([85,255,255])
# Threshold the HSV image to get only blue colors
mask = cv.inRange(hsv, lower, upper)
# Bitwise-AND mask and original image
res = cv.bitwise_and(frame,frame, mask= mask)
# cv.imshow('frame',frame)
cv.imshow('mask',mask)
cv.imshow('res',res)
cv.waitKey(0) 
cv.destroyAllWindows()

In [98]:
# convert to bgr and sharpen image
green_img = cv.cvtColor(res, cv.COLOR_HSV2BGR)
green_sharped = sharpImg(green_img)

In [99]:
# Let's define our kernel size
kernel = np.ones((6,6), np.uint8)
closing = cv.morphologyEx(green_sharped, cv.MORPH_CLOSE, kernel)
cv.imshow('Closing', closing)
cv.waitKey(0)
cv.destroyAllWindows()

In [100]:

dilation = cv.dilate(green_sharped, kernel, iterations = 5)
# cv.imshow('Dilation', dilation)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [101]:
green_edges = cv.Canny(dilation, 50, 120)
cv.imshow('green_edges', green_edges)
cv.waitKey(0)
cv.destroyWindow('green_edges')

### Get edge image

In [8]:
import numpy as np
# Create our shapening kernel, we don't normalize since the 
# the values in the matrix sum to 1
kernel_sharpening = np.array([[-1,-1,-1], 
                              [-1,9,-1], 
                              [-1,-1,-1]])

# applying different kernels to the input image
sharpened = cv.filter2D(target_gray, -1, kernel_sharpening)

cv.imshow('Image Sharpening', sharpened)

cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
target_edges = cv.Canny(sharpened, 100, 150)
cv.imshow('target_edges', target_edges)
cv.waitKey(0)
cv.destroyWindow('target_edges')

In [ ]:
print(target.shape)

### Find the rectangel bounding the card

In [104]:
sharpened = green_edges
# ret, thresh = cv.threshold(sharpened, 127, 255, 1)

# Extract Contours
# contours, hierarchy = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL,
#                                       cv.CHAIN_APPROX_NONE)

contours, hierarchy = cv.findContours(green_edges, cv.RETR_EXTERNAL,
                                      cv.CHAIN_APPROX_SIMPLE)

contours = sorted(contours, key=cv.contourArea, reverse=True)

imgShow = target.copy()
for c in contours[:5]:
    cv.drawContours(imgShow, [c], -1, (0, 40, 120), 2)
    cv.imshow('Template_Contours', imgShow)
    cv.waitKey(1)
    if cv.waitKey(1) == 13: #13 is the Enter Key
        break

cv.waitKey(0)
cv.destroyAllWindows()

In [105]:
img = target.copy()
x,y,w,h
for cnt in contours:
    # Get approximate polygons
    approx = cv.approxPolyDP(cnt, 0.02 * cv.arcLength(cnt, True), True)
    if len(approx) == 4:
        x, y, w, h = cv.boundingRect(cnt)
        M = cv.moments(cnt)
        print(M)
        # continue
        cx = int(M['m10'] / M['m00'])
        cy = int(M['m01'] / M['m00'])

        shape_name = "Rectangle"
        # Find contour center to place text at the center
        cv.drawContours(img, [cnt], 0, (0, 0, 255), 2)
        cv.putText(img, shape_name, (cx - 50, cy), cv.FONT_HERSHEY_SIMPLEX, 1,
                   (0, 0, 0), 1)
        break

cv.imshow('Identifying Shapes', img)
cv.waitKey(0)
cv.destroyAllWindows()

{'m00': 820595.0, 'm10': 482205139.5, 'm01': 371153864.6666666, 'm20': 369148349422.6666, 'm11': 217756024054.41666, 'm02': 204556658605.16666, 'm30': 317260743656519.94, 'm21': 166506074775035.53, 'm12': 120240918771325.14, 'm03': 125731335077120.2, 'mu20': 85790789895.47662, 'mu11': -344623753.03292847, 'mu02': 36684332723.33423, 'mu30': -487659573183.75, 'mu21': -54138038509.8125, 'mu12': 349309362578.98047, 'mu03': 26135666236.3125, 'nu20': 0.12740396193520617, 'nu11': -0.0005117849079938372, 'nu02': 0.054478217715402316, 'nu30': -0.000799456040260556, 'nu21': -8.875245001746316e-05, 'nu12': 0.0005726484112885576, 'nu03': 4.284611107384316e-05}


In [106]:
cv.imshow('card1', target)
cv.imshow('card2', target[y:y+h+1,x:x+w+1])

cv.waitKey(0)
cv.destroyAllWindows()

In [29]:
cv.destroyAllWindows()